In [39]:
%%pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *

orderSchema = StructType([
     StructField("SalesOrderNumber", StringType()),
     StructField("SalesOrderLineNumber", IntegerType()),
     StructField("OrderDate", DateType()),
     StructField("CustomerName", StringType()),
     StructField("Email", StringType()),
     StructField("Item", StringType()),
     StructField("Quantity", IntegerType()),
     StructField("UnitPrice", FloatType()),
     StructField("Tax", FloatType())
    ])


df = spark.read.load('abfss://files@datalakeuyld2gn.dfs.core.windows.net/sales/orders/2021.csv', format='csv', schema=orderSchema
##, header=True
)
display(df.limit(100))

In [40]:
df.printSchema()

In [41]:
customers = df.select('CustomerName', 'Email').where(df['Item']=='Road-250 Red, 52')
print(customers.count())
print(customers.distinct().count())
display(customers.distinct())

In [42]:
productSales = df.select("Item", "Quantity").groupBy("Item").sum()
display(productSales)

In [43]:
yearlySales = df.select(year("OrderDate").alias("Year")).groupby("Year").count().orderBy("Year")
display(yearlySales)


'''  
Run the code cell you added, and note that the results show the number of sales orders per year.    /n
Note that the select method includes a SQL year function to extract the year component of           /n
the OrderDate field, and then an alias method is used to assign a columm name to the extracted      /n
year value. The data is then grouped by the derived Year column and the count of rows in each       /n
group is calculated before finally the orderBy method is used to sort the resulting dataframe.
'''

In [44]:
df.createOrReplaceTempView("salesorders")
spark_df = spark.sql("select * from salesorders")
display(spark_df)

In [45]:
%%sql
select year(OrderDate) as OrderYear,
    sum((UnitPrice * Quantity) + Tax) as GrossRevenue
from salesorders
group by year(OrderDate)
order by OrderYear;

In [46]:
%%sql

select * from salesorders

In [47]:
 sqlQuery = "SELECT CAST(YEAR(OrderDate) AS CHAR(4)) AS OrderYear, \
                 SUM((UnitPrice * Quantity) + Tax) AS GrossRevenue \
             FROM salesorders \
             GROUP BY CAST(YEAR(OrderDate) AS CHAR(4)) \
             ORDER BY OrderYear"
 df_spark = spark.sql(sqlQuery)
 df_spark.show()


In [48]:
from matplotlib import pyplot as plt

df_sales = df_spark.toPandas()

plt.bar(x=df_sales['OrderYear'], height=df_sales['GrossRevenue'])

plt.show()

In [49]:
 # Clear the plot area
 plt.clf()

 # Create a bar plot of revenue by year
 plt.bar(x=df_sales['OrderYear'], height=df_sales['GrossRevenue'], color='orange')

 # Customize the chart
 plt.title('Revenue by Year')
 plt.xlabel('Year')
 plt.ylabel('Revenue')
 plt.grid(color='#95a5a6', linestyle='--', linewidth=2, axis='y', alpha=0.7)
 plt.xticks(rotation=45)

 # Show the figure
 plt.show()


In [50]:
 # Clear the plot area
 plt.clf()

 # Create a Figure
 fig = plt.figure(figsize=(8,3))

 # Create a bar plot of revenue by year
 plt.bar(x=df_sales['OrderYear'], height=df_sales['GrossRevenue'], color='orange')

 # Customize the chart
 plt.title('Revenue by Year')
 plt.xlabel('Year')
 plt.ylabel('Revenue')
 plt.grid(color='#95a5a6', linestyle='--', linewidth=2, axis='y', alpha=0.7)
 plt.xticks(rotation=45)

 # Show the figure
 plt.show()


In [51]:
 # Clear the plot area
 plt.clf()

 # Create a figure for 2 subplots (1 row, 2 columns)
 fig, ax = plt.subplots(1, 2, figsize = (10,4))

 # Create a bar plot of revenue by year on the first axis
 ax[0].bar(x=df_sales['OrderYear'], height=df_sales['GrossRevenue'], color='orange')
 ax[0].set_title('Revenue by Year')

 # Create a pie chart of yearly order counts on the second axis
 yearly_counts = df_sales['OrderYear'].value_counts()
 ax[1].pie(yearly_counts)
 ax[1].set_title('Orders per Year')
 ax[1].legend(yearly_counts.keys().tolist())

 # Add a title to the Figure
 fig.suptitle('Sales Data')

 # Show the figure
 plt.show()


In [52]:
 import seaborn as sns

 # Clear the plot area
 plt.clf()

 # Create a bar chart
 ax = sns.barplot(x="OrderYear", y="GrossRevenue", data=df_sales)
 plt.show()


In [53]:
 # Clear the plot area
 plt.clf()

 # Set the visual theme for seaborn
 sns.set_theme(style="whitegrid")

 # Create a bar chart
 ax = sns.barplot(x="OrderYear", y="GrossRevenue", data=df_sales)
 plt.show()


In [54]:
 # Clear the plot area
 plt.clf()

 # Create a bar chart
 ax = sns.lineplot(x="OrderYear", y="GrossRevenue", data=df_sales)
 plt.show()
